In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, GridSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [17]:
df = pd.read_csv("data.csv")

In [3]:
df["target_up"] = (df["price"].shift(-14) > df["price"]).astype(int)
df = df.iloc[:-14]   # retirer la dernière ligne sans cible

In [4]:
#choix de la cible  

target = df['target_up']
target.shape[0]

334

In [5]:
dataexplicative = df.drop(columns=['date', 'target_up'])

In [19]:
df.shape[0]

337

In [7]:
dataexplicative.shape[0]

334

In [8]:
dataexplicative.tail()

,price,volume,return_1d,return_3d,return_7d,return_14d,MA7,MA30,MA_diff,vol_7d,vol_MA7,highest_30,lowest_30,pos_channel_30
329,85762.206093,8.639245e+10,-2.079255,-6.401507,-2.251372,-8.316363,88905.340674,92658.963301,-3753.622627,2.130685,6.254074e+10,98422.812122,85762.206093,0.000000
330,81256.301578,9.737908e+10,-5.253951,-8.561380,-9.078764,-13.985412,87746.242070,92198.806835,-4452.564766,2.575709,6.489116e+10,98422.812122,81256.301578,0.000000
331,82171.816288,3.360760e+10,1.126700,-6.178655,-7.060250,-13.405739,86854.491413,91858.415534,-5003.924121,2.740059,6.338302e+10,98422.812122,81256.301578,0.053331
332,81307.886643,6.256266e+10,-1.051370,-5.193802,-10.370709,-15.292969,85510.510417,91525.824561,-6015.314145,2.239811,6.490053e+10,98422.812122,81256.301578,0.003005
333,79409.746949,8.265206e+10,-2.334509,-2.272506,-13.334405,-14.128123,83765.079436,91108.675922,-7343.596486,1.946201,6.835965e+10,98422.812122,79409.746949,0.000000


In [9]:
df.tail()

,date,price,volume,return_1d,return_3d,return_7d,return_14d,MA7,MA30,MA_diff,vol_7d,vol_MA7,highest_30,lowest_30,pos_channel_30,target_up
329,2025-11-14,85762.206093,8.639245e+10,-2.079255,-6.401507,-2.251372,-8.316363,88905.340674,92658.963301,-3753.622627,2.130685,6.254074e+10,98422.812122,85762.206093,0.000000,0
330,2025-11-15,81256.301578,9.737908e+10,-5.253951,-8.561380,-9.078764,-13.985412,87746.242070,92198.806835,-4452.564766,2.575709,6.489116e+10,98422.812122,81256.301578,0.000000,0
331,2025-11-16,82171.816288,3.360760e+10,1.126700,-6.178655,-7.060250,-13.405739,86854.491413,91858.415534,-5003.924121,2.740059,6.338302e+10,98422.812122,81256.301578,0.053331,0
332,2025-11-17,81307.886643,6.256266e+10,-1.051370,-5.193802,-10.370709,-15.292969,85510.510417,91525.824561,-6015.314145,2.239811,6.490053e+10,98422.812122,81256.301578,0.003005,0
333,2025-11-18,79409.746949,8.265206e+10,-2.334509,-2.272506,-13.334405,-14.128123,83765.079436,91108.675922,-7343.596486,1.946201,6.835965e+10,98422.812122,79409.746949,0.000000,1


In [10]:
target.tail()

329    0
330    0
331    0
332    0
333    1
Name: target_up, dtype: int32

In [11]:

# Variables explicatives et cible
X = dataexplicative
#X = X.drop(columns=['NiveauSiteEUIWN'])
y = target

# Encodage One-Hot basé sur X
X = pd.get_dummies(X, drop_first=True)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)



# Normalisation des données 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # OK : on apprend la normalisation sur l'entraînement
X_test_scaled = scaler.transform(X_test)        # OK : on applique la même normalisation au test

# Appliquer le modèle DummyRegressor
dummy_regressor = DummyClassifier(strategy="stratified")  # 'mean' pour prédire la moyenne des cibles
dummy_regressor.fit(X_train, y_train)

# Faire des prédictions
y_pred = dummy_regressor.predict(X_test)

# Calculer les erreurs
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Vérifier si des valeurs de test sont déjà vues en entraînement
intersection = set(X_train.index).intersection(set(X_test.index))
print(f"🔍 Nombre de valeurs identiques dans Train/Test : {len(intersection)}")

# Accuracy globale
accuracy = accuracy_score(y_test, y_pred)

# Precision par classe
precision_baisse = precision_score(y_test, y_pred, pos_label=0)
precision_hausse = precision_score(y_test, y_pred, pos_label=1)

# Recall par classe
recall_baisse = recall_score(y_test, y_pred, pos_label=0)
recall_hausse = recall_score(y_test, y_pred, pos_label=1)

# F1-score par classe
f1_baisse = f1_score(y_test, y_pred, pos_label=0)
f1_hausse = f1_score(y_test, y_pred, pos_label=1)

print("📊 Accuracy globale :", accuracy)
print("\n🔵 Classe BAISSE (0) :")
print("Precision :", precision_baisse)
print("Recall    :", recall_baisse)
print("F1-score  :", f1_baisse)

print("\n🟢 Classe HAUSSE (1) :")
print("Precision :", precision_hausse)
print("Recall    :", recall_hausse)
print("F1-score  :", f1_hausse)

# Affichage détaillé :
print("\n📄 Classification report :")
print(classification_report(y_test, y_pred, target_names=["baisse","hausse"]))

# Matrice de confusion :
print("🔢 Matrice de confusion :")
print(confusion_matrix(y_test, y_pred))

🔍 Nombre de valeurs identiques dans Train/Test : 0
📊 Accuracy globale : 0.47761194029850745

🔵 Classe BAISSE (0) :
Precision : 0.5142857142857142
Recall    : 0.5
F1-score  : 0.5070422535211268

🟢 Classe HAUSSE (1) :
Precision : 0.4375
Recall    : 0.45161290322580644
F1-score  : 0.4444444444444444

📄 Classification report :
              precision    recall  f1-score   support

      baisse       0.51      0.50      0.51        36
      hausse       0.44      0.45      0.44        31

    accuracy                           0.48        67
   macro avg       0.48      0.48      0.48        67
weighted avg       0.48      0.48      0.48        67

🔢 Matrice de confusion :
[[18 18]
 [17 14]]


In [12]:
# Variables explicatives et cible
X = dataexplicative
#X = X.drop(columns=['NiveauSiteEUIWN'])
y = target


X = pd.get_dummies(X, drop_first=True)  # Conversion des variables catégorielles en variables binaires

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


# 🔹 Normalisation des variables explicatives uniquement
scaler = StandardScaler()

# Appliquer le scaler uniquement aux variables explicatives
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # Appliquer la même transformation sur les données de test



logreg = LogisticRegression(
    penalty="elasticnet",
    l1_ratio=0.5,  # mélange L1/L2
    solver="saga"
)
# 🔹 Création du modèle ElasticNet avec alpha fixé à 1
elasticnet = logreg

# Entraînement du modèle
elasticnet.fit(X_train_scaled, y_train)

# 🔹 Faire des prédictions sur l'ensemble de test
y_pred = elasticnet.predict(X_test_scaled)

# 🔹 Évaluer le modèle sur l'ensemble de test
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# Affichage des résultats
print(f"📊 R² Score: {r2:.4f}")
print(f"📉 Mean Squared Error: {mse:.4f}")

# Faire des prédictions sur l'ensemble d'entraînement
y_pred_train = elasticnet.predict(X_train_scaled)
# Accuracy globale
accuracy = accuracy_score(y_test, y_pred)

# Precision par classe
precision_baisse = precision_score(y_test, y_pred, pos_label=0)
precision_hausse = precision_score(y_test, y_pred, pos_label=1)

# Recall par classe
recall_baisse = recall_score(y_test, y_pred, pos_label=0)
recall_hausse = recall_score(y_test, y_pred, pos_label=1)

# F1-score par classe
f1_baisse = f1_score(y_test, y_pred, pos_label=0)
f1_hausse = f1_score(y_test, y_pred, pos_label=1)

print("📊 Accuracy globale :", accuracy)
print("\n🔵 Classe BAISSE (0) :")
print("Precision :", precision_baisse)
print("Recall    :", recall_baisse)
print("F1-score  :", f1_baisse)

print("\n🟢 Classe HAUSSE (1) :")
print("Precision :", precision_hausse)
print("Recall    :", recall_hausse)
print("F1-score  :", f1_hausse)

# Affichage détaillé :
print("\n📄 Classification report :")
print(classification_report(y_test, y_pred, target_names=["baisse","hausse"]))



📊 R² Score: -1.0412
📉 Mean Squared Error: 0.5075
📊 Accuracy globale : 0.4925373134328358

🔵 Classe BAISSE (0) :
Precision : 0.55
Recall    : 0.3055555555555556
F1-score  : 0.39285714285714285

🟢 Classe HAUSSE (1) :
Precision : 0.46808510638297873
Recall    : 0.7096774193548387
F1-score  : 0.5641025641025641

📄 Classification report :
              precision    recall  f1-score   support

      baisse       0.55      0.31      0.39        36
      hausse       0.47      0.71      0.56        31

    accuracy                           0.49        67
   macro avg       0.51      0.51      0.48        67
weighted avg       0.51      0.49      0.47        67



In [13]:

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split



# Variables explicatives et cible
X = dataexplicative
y = target



# 🎲 Séparation des données avec stratification basée sur les quartiles de la cible
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2,  random_state=42#, stratify = y_quartiles  # Stratification sur y_quartiles
)

# 🔧 Pipeline de prétraitement
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X_train.select_dtypes(exclude='object').columns.tolist()),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), X_train.select_dtypes(include='object').columns.tolist())
    ]
)

# 💡 Pipeline complet avec GridSearchCV
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', LogisticRegression(
        penalty='elasticnet',
        solver='saga',
        max_iter=1000,
        random_state=42
    ))
])

param_grid = {
    'model__C': [0.01, 0.1, 1, 10],
    'model__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    scoring='f1_macro',
    cv=5,
    verbose=1,
    n_jobs=-1
)

# 🚀 Entraînement avec GridSearchCV
grid_search.fit(X_train, y_train)

# ✅ Meilleur modèle trouvé
best_model = grid_search.best_estimator_

print(f"✅ Meilleur alpha trouvé : {grid_search.best_params_}")
print(f"📊 Meilleur F1_macro (validation croisée) : {grid_search.best_score_:.4f}")

# 🟢 Prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test)

# 📊 Évaluation globale
accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_weighted = f1_score(y_test, y_pred, average='weighted')

print("📊 Évaluation sur TEST")
print(f"🎯 Accuracy        : {accuracy:.4f}")
print(f"💠 F1 Macro        : {f1_macro:.4f}")
print(f"🔷 F1 Weighted     : {f1_weighted:.4f}")

# 📊 Rapport détaillé par classe
print("\n🔍 Rapport détaillé par classe :")
print(classification_report(y_test, y_pred, target_names=['BAISSE', 'HAUSSE']))

# 🧩 Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
print("\n🧩 Matrice de confusion :")
print(cm)

# 🔵 Évaluation entraînement (pour voir l’overfitting)
y_pred_train = best_model.predict(X_train)

accuracy_train = accuracy_score(y_train, y_pred_train)
f1_macro_train = f1_score(y_train, y_pred_train, average='macro')

print("\n📊 Évaluation sur TRAIN")
print(f"🎯 Accuracy (train) : {accuracy_train:.4f}")
print(f"💠 F1 Macro (train) : {f1_macro_train:.4f}")

# 📈 Extraire les coefficients du modèle
coefs = best_model.named_steps['model'].coef_.flatten()
features = best_model.named_steps['preprocessing'].get_feature_names_out()

coefficients = pd.DataFrame({
    'Feature': features,
    'Coefficient': coefs
}).sort_values(by='Coefficient', ascending=False)

print("\n📈 Coefficients du modèle :")
print(coefficients)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
✅ Meilleur alpha trouvé : {'model__C': 1, 'model__l1_ratio': 0.3}
📊 Meilleur F1_macro (validation croisée) : 0.4746
📊 Évaluation sur TEST
🎯 Accuracy        : 0.4925
💠 F1 Macro        : 0.4785
🔷 F1 Weighted     : 0.4721

🔍 Rapport détaillé par classe :
              precision    recall  f1-score   support

      BAISSE       0.55      0.31      0.39        36
      HAUSSE       0.47      0.71      0.56        31

    accuracy                           0.49        67
   macro avg       0.51      0.51      0.48        67
weighted avg       0.51      0.49      0.47        67


🧩 Matrice de confusion :
[[11 25]
 [ 9 22]]

📊 Évaluation sur TRAIN
🎯 Accuracy (train) : 0.5993
💠 F1 Macro (train) : 0.5597

📈 Coefficients du modèle :
                Feature  Coefficient
13  num__pos_channel_30     0.129052
1           num__volume     0.058282
4        num__return_7d     0.049754
11      num__highest_30     0.015839
0            num__pri

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder  # Nous utilisons category_encoders pour le Target Encoding

# Variables explicatives et cible
X = dataexplicative
y = target



# 🎲 Séparation des données avec stratification basée sur les quartiles de la cible
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42#, stratify=y_quartiles  # Stratification sur y_quartiles
)

# 🔧 Pipeline de prétraitement
# Nous remplaçons le OneHotEncoder par un TargetEncoder pour les variables catégorielles
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X_train.select_dtypes(exclude='object').columns.tolist()),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), X_train.select_dtypes(include='object').columns.tolist())
    ]
)

# 💡 Pipeline complet avec GridSearchCV
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestClassifier(random_state=42))  # Modèle ElasticNet
])

# Paramètres pour GridSearchCV
param_grid = {
    'model__max_depth': [10, 5, 20, 30],  # Using model__ to specify RandomForestRegressor
    'model__n_estimators': [10, 50, 100, 200],  # Add more parameters for the model if needed
    'model__min_samples_split': [3, 2, 4, 5, 6]
    # Add other hyperparameters for preprocessing steps if needed
}

# GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    scoring='f1_macro',
    cv=5,
    verbose=1,
    n_jobs=-1
)

# 🚀 Entraînement avec GridSearchCV
grid_search.fit(X_train, y_train)

# ✅ Meilleur modèle trouvé
best_model = grid_search.best_estimator_

print(f"✅ Meilleur alpha trouvé : {grid_search.best_params_}")
print(f"📊 Meilleur f1_macro (validation croisée) : {grid_search.best_score_:.4f}")

# 🟢 Prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_weighted = f1_score(y_test, y_pred, average='weighted')

print("📊 Évaluation sur TEST")
print(f"🎯 Accuracy        : {accuracy:.4f}")
print(f"💠 F1 Macro        : {f1_macro:.4f}")
print(f"🔷 F1 Weighted     : {f1_weighted:.4f}")

# 📊 Rapport détaillé par classe
print("\n🔍 Rapport détaillé par classe :")
print(classification_report(y_test, y_pred, target_names=['BAISSE', 'HAUSSE']))

# 🧩 Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
print("\n🧩 Matrice de confusion :")
print(cm)

# 🔵 Évaluation entraînement (pour voir l’overfitting)
y_pred_train = best_model.predict(X_train)

accuracy_train = accuracy_score(y_train, y_pred_train)
f1_macro_train = f1_score(y_train, y_pred_train, average='macro')

print("\n📊 Évaluation sur TRAIN")
print(f"🎯 Accuracy (train) : {accuracy_train:.4f}")
print(f"💠 F1 Macro (train) : {f1_macro_train:.4f}")

# 🔵 Prédictions TEST
y_pred_test = best_model.predict(X_test)

# 📊 Évaluation TEST
accuracy_test = accuracy_score(y_test, y_pred_test)
f1_macro_test = f1_score(y_test, y_pred_test, average='macro')

print("\n📊 Évaluation sur TEST")
print(f"🎯 Accuracy (test) : {accuracy_test:.4f}")
print(f"💠 F1 Macro (test) : {f1_macro_test:.4f}")


Fitting 5 folds for each of 80 candidates, totalling 400 fits
✅ Meilleur alpha trouvé : {'model__max_depth': 10, 'model__min_samples_split': 2, 'model__n_estimators': 200}
📊 Meilleur f1_macro (validation croisée) : 0.6414
📊 Évaluation sur TEST
🎯 Accuracy        : 0.5522
💠 F1 Macro        : 0.5398
🔷 F1 Weighted     : 0.5342

🔍 Rapport détaillé par classe :
              precision    recall  f1-score   support

      BAISSE       0.65      0.36      0.46        36
      HAUSSE       0.51      0.77      0.62        31

    accuracy                           0.55        67
   macro avg       0.58      0.57      0.54        67
weighted avg       0.59      0.55      0.53        67


🧩 Matrice de confusion :
[[13 23]
 [ 7 24]]

📊 Évaluation sur TRAIN
🎯 Accuracy (train) : 1.0000
💠 F1 Macro (train) : 1.0000

📊 Évaluation sur TEST
🎯 Accuracy (test) : 0.5522
💠 F1 Macro (test) : 0.5398


In [15]:
#decisiontree
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder  # Nous utilisons category_encoders pour le Target Encoding

# Variables explicatives et cible
X = dataexplicative
y = target


# 🎲 Séparation des données avec stratification basée sur les quartiles de la cible
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42#, stratify=y_quartiles  # Stratification sur y_quartiles
)

# 🔧 Pipeline de prétraitement
# Nous remplaçons le OneHotEncoder par un TargetEncoder pour les variables catégorielles
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X_train.select_dtypes(exclude='object').columns.tolist()),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), X_train.select_dtypes(include='object').columns.tolist())
    ]
)

# 💡 Pipeline complet avec GridSearchCV
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', DecisionTreeClassifier(random_state=42))  # Modèle ElasticNet
])


param_grid = {
    'model__max_depth': [10, 5, 20, 30],  # Using model__ to specify RandomForestRegressor
    'model__min_samples_leaf': [2, 3, 4, 5, 6],  # Add more parameters for the model if needed
    'model__min_samples_split': [3, 2, 4]
    # Add other hyperparameters for preprocessing steps if needed
}

# GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    scoring='f1_macro',
    cv=5,
    verbose=1,
    n_jobs=-1
)

# 🚀 Entraînement avec GridSearchCV
grid_search.fit(X_train, y_train)

# ✅ Meilleur modèle trouvé
best_model = grid_search.best_estimator_

print(f"✅ Meilleur alpha trouvé : {grid_search.best_params_}")
print(f"📊 Meilleur f1_macro (validation croisée) : {grid_search.best_score_:.4f}")

# 🟢 Prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test)

print(f"✅ Meilleur alpha trouvé : {grid_search.best_params_}")
print(f"📊 Meilleur f1_macro (validation croisée) : {grid_search.best_score_:.4f}")

# 🟢 Prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_weighted = f1_score(y_test, y_pred, average='weighted')

print("📊 Évaluation sur TEST")
print(f"🎯 Accuracy        : {accuracy:.4f}")
print(f"💠 F1 Macro        : {f1_macro:.4f}")
print(f"🔷 F1 Weighted     : {f1_weighted:.4f}")

# 📊 Rapport détaillé par classe
print("\n🔍 Rapport détaillé par classe :")
print(classification_report(y_test, y_pred, target_names=['BAISSE', 'HAUSSE']))

# 🧩 Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
print("\n🧩 Matrice de confusion :")
print(cm)

# 🔵 Évaluation entraînement (pour voir l’overfitting)
y_pred_train = best_model.predict(X_train)

accuracy_train = accuracy_score(y_train, y_pred_train)
f1_macro_train = f1_score(y_train, y_pred_train, average='macro')

print("\n📊 Évaluation sur TRAIN")
print(f"🎯 Accuracy (train) : {accuracy_train:.4f}")
print(f"💠 F1 Macro (train) : {f1_macro_train:.4f}")


Fitting 5 folds for each of 60 candidates, totalling 300 fits
✅ Meilleur alpha trouvé : {'model__max_depth': 20, 'model__min_samples_leaf': 2, 'model__min_samples_split': 3}
📊 Meilleur f1_macro (validation croisée) : 0.5715
✅ Meilleur alpha trouvé : {'model__max_depth': 20, 'model__min_samples_leaf': 2, 'model__min_samples_split': 3}
📊 Meilleur f1_macro (validation croisée) : 0.5715
📊 Évaluation sur TEST
🎯 Accuracy        : 0.6119
💠 F1 Macro        : 0.6077
🔷 F1 Weighted     : 0.6046

🔍 Rapport détaillé par classe :
              precision    recall  f1-score   support

      BAISSE       0.71      0.47      0.57        36
      HAUSSE       0.56      0.77      0.65        31

    accuracy                           0.61        67
   macro avg       0.63      0.62      0.61        67
weighted avg       0.64      0.61      0.60        67


🧩 Matrice de confusion :
[[17 19]
 [ 7 24]]

📊 Évaluation sur TRAIN
🎯 Accuracy (train) : 0.9588
💠 F1 Macro (train) : 0.9586


In [16]:
#gradientboosting
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder  # Nous utilisons category_encoders pour le Target Encoding

# Variables explicatives et cible
X = dataexplicative
y = target



# 🎲 Séparation des données avec stratification basée sur les quartiles de la cible
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42#, stratify=y_quartiles  # Stratification sur y_quartiles
)

# 🔧 Pipeline de prétraitement
# Nous remplaçons le OneHotEncoder par un TargetEncoder pour les variables catégorielles
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X_train.select_dtypes(exclude='object').columns.tolist()),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), X_train.select_dtypes(include='object').columns.tolist())
    ]
)

# 💡 Pipeline complet avec GridSearchCV
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', GradientBoostingClassifier(random_state=42))  # Modèle ElasticNet
])

param_grid = {
    'model__max_depth': [10, 5, 2, 3],  # Using model__ to specify RandomForestRegressor
    'model__n_estimators': [100, 200, 300, 50],  # Add more parameters for the model if needed
    'model__learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2, 0.3],
    # Add other hyperparameters for preprocessing steps if needed
}

# GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    scoring='f1_macro',
    cv=5,
    verbose=1,
    n_jobs=-1
)

# 🚀 Entraînement avec GridSearchCV
grid_search.fit(X_train, y_train)

# ✅ Meilleur modèle trouvé
best_model = grid_search.best_estimator_

print(f"✅ Meilleur alpha trouvé : {grid_search.best_params_}")
print(f"📊 Meilleur f1_macro (validation croisée) : {grid_search.best_score_:.4f}")

# 🟢 Prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_weighted = f1_score(y_test, y_pred, average='weighted')

print("📊 Évaluation sur TEST")
print(f"🎯 Accuracy        : {accuracy:.4f}")
print(f"💠 F1 Macro        : {f1_macro:.4f}")
print(f"🔷 F1 Weighted     : {f1_weighted:.4f}")

# 📊 Rapport détaillé par classe
print("\n🔍 Rapport détaillé par classe :")
print(classification_report(y_test, y_pred, target_names=['BAISSE', 'HAUSSE']))

# 🧩 Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
print("\n🧩 Matrice de confusion :")
print(cm)

# 🔵 Évaluation entraînement (pour voir l’overfitting)
y_pred_train = best_model.predict(X_train)

accuracy_train = accuracy_score(y_train, y_pred_train)
f1_macro_train = f1_score(y_train, y_pred_train, average='macro')

print("\n📊 Évaluation sur TRAIN")
print(f"🎯 Accuracy (train) : {accuracy_train:.4f}")
print(f"💠 F1 Macro (train) : {f1_macro_train:.4f}")


Fitting 5 folds for each of 96 candidates, totalling 480 fits


KeyboardInterrupt: 